In [4]:
from audiolm_pytorch import AudioLMSoundStream

In [6]:
soundstream = AudioLMSoundStream() # say you want the hyperparameters as in Audio LM paper

AssertionError: if use_finite_scalar_quantizer is set to False, `codebook_size` must be set (and not `finite_scalar_quantizer_levels`)